# uGAN2D_cov19_lungles
* filtre = 8
* 2 channels
* n_patch = 6
* lr = dynamic
* alpha = 5

# IMPORTS & INSTALL

In [ ]:
! pip install elasticdeform
! pip install tensorflow_addons
! pip install gdown


In [ ]:
import gdown

inputs_file = 'https://drive.google.com/file/d/1-XPgETUZYwspdQBaiNk7K7XR7yeyYgcA/view?usp=sharing'
inputs_url = 'https://drive.google.com/uc?id=1-XPgETUZYwspdQBaiNk7K7XR7yeyYgcA'

inputs_output = 'inputs.zip'

gdown.download(inputs_url, inputs_output, quiet=False)

In [ ]:
! unzip ./inputs.zip -d ./INPUTS
! mv -v ./INPUTS/content/drive/MyDrive/GANs/CovidLesionDetection/INPUTS/* ./INPUTS
! rm -rf ./INPUTS/content
! rm ./inputs.zip

In [ ]:
EXEC = 6

VERSION = 'uGAN2D_cov19_lungles'

INPUT_PATH = './INPUTS'

PATH = './RESULTS' 

DATA_PATH = './data'

In [ ]:
if EXEC!=1:
    results_file = 'https://drive.google.com/file/d/156AkeU7wx1n4vmcYZ30ulCeaY0Nmq8V0/view?usp=sharing'
    results_url = 'https://drive.google.com/uc?id=156AkeU7wx1n4vmcYZ30ulCeaY0Nmq8V0'

    results_output = 'results.zip'

    gdown.download(results_url, results_output, quiet=False)
    
else:
    ! mkdir ./RESULTS

In [ ]:
if EXEC!=1:
    ! unzip ./results.zip -d ./RESULTS
    ! mv -v ./RESULTS/content/drive/MyDrive/GANs/CovidLesionDetection/RESULTS_KAGGLE/VERSION_$VERSION/RESULTS_{EXEC-1}/* ./RESULTS
    ! rm -rf ./RESULTS/content
    ! rm ./results.zip  

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn
import cv2 as cv
import nibabel as nib
import pickle
import imgaug as ia
import imgaug.augmenters as iaa
import tqdm
import gc
import warnings
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras import losses, metrics
from keras import optimizers
from keras import callbacks
from keras.models import Model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers import concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers import Multiply

import math
from functools import reduce
import glob
import time
import os 
from tensorflow.keras.utils import to_categorical
from sys import stdout
from scipy.ndimage.interpolation import affine_transform
from sklearn.model_selection import train_test_split
import elasticdeform
import matplotlib.image as mpim
import multiprocessing as mp
import json

In [ ]:
# metadata = pd.read_csv('/content/drive/MyDrive/COVID19_Dataset/metadata.csv')
# # metadata.replace('../input/covid19-ct-scans/', '', regex=True, inplace=True)
# print(metadata.shape)
# metadata.head()

# PATCH EXTRACTION AND AUGMENTATION

In [ ]:
#PATCH EXTRACTION AND AUGMENTATION technique

def patch_extraction(Xb, yb, PatchesShape=(128,128,32), Npatches=1, max_tries = 1000, background_threshold=0.95, num_classes=4, apply_ratio=False):
    """
    3D patch extraction
    """
    
    batch_size = len(Xb)
    X_patches = np.empty((batch_size*Npatches, PatchesShape[0], PatchesShape[1], PatchesShape[2]))
    y_patches = np.empty((batch_size*Npatches, PatchesShape[0], PatchesShape[1], PatchesShape[2]))
    i = 0
    
    if apply_ratio:
#       print(f'Application of ratio(bg_ratio < {background_threshold*100}%): {apply_ratio}')
      for b in range(batch_size):
          Xb_temp = Xb[b]
          yb_temp = yb[b]
          rows, columns, slices= Xb_temp.shape
          for p in range(Npatches):
              
              temp_X = None
              temp_y = None

              tries = 0
              out = False    
              inf = 1

              while (tries < max_tries) and (not out):
                
                x = np.random.randint(rows-PatchesShape[0]+1) 
                y = np.random.randint(columns-PatchesShape[1]+1)
                z = np.random.randint(slices-PatchesShape[2]+1)  

                
                temp_X = Xb_temp[x:x+PatchesShape[0], y:y+PatchesShape[1], z:z+PatchesShape[2]]
               
                temp_y = yb_temp[x:x+PatchesShape[0], y:y+PatchesShape[1], z:z+PatchesShape[2]]

                
                bgrd_ratio = 1 - np.sum(temp_y[temp_y!=0])/(PatchesShape[0] * PatchesShape[1] * PatchesShape[2])
                bgrd_ratio_off = bgrd_ratio

                if bgrd_ratio <= inf :
                  temp_x_min = temp_X
                  temp_y_min = temp_y
                  inf = bgrd_ratio
                  
                tries += 1

                if bgrd_ratio < background_threshold:
                  out = True
                  temp_X = Xb_temp[x:x+PatchesShape[0], y:y+PatchesShape[1], z:z+PatchesShape[2]]
                else :
                  temp_X = temp_x_min
                  temp_y = temp_y_min
                  bgrd_ratio_off = inf
                
#               print(f'\nBG_ratio : {bgrd_ratio_off*100} % for index {i}')
#               print(f"\nTried {tries} times to find a sub-volume. Exit...")
              X_patches[i] = temp_X
              y_patches[i] = temp_y
              i += 1
      
      del Xb_temp, yb_temp, temp_X, temp_y,temp_x_min,temp_y_min
    
    else:
      for b in range(batch_size):
        Xb_temp = Xb[b]
        yb_temp = yb[b]
        rows, columns, slices= Xb_temp.shape
        
        if rows < PatchesShape[0]:
            if (rows%2)!=0:
                rows = rows-1
            
            new_frame_X = np.empty((PatchesShape[0],columns,slices))
            new_frame_y = np.empty((PatchesShape[0],columns,slices))
            
            for slc in range(slices):
                slicee_X = Xb_temp[:,:,slc]
                slicee_y = yb_temp[:,:,slc]
                
                for c in range(columns):
                    column_X = slicee_X[:,c]
                    column_y = slicee_y[:,c]
                    pad_arr_X = np.pad(column_X[:-1], (((PatchesShape[0]-rows)//2),), 'constant', constant_values=(0, 0))
                    pad_arr_y = np.pad(column_y[:-1], (((PatchesShape[0]-rows)//2),), 'constant', constant_values=(0, 0))

                    new_frame_X[:,c,slc] = pad_arr_X
                    new_frame_y[:,c,slc] = pad_arr_y
            
            Xb_temp = new_frame_X
            yb_temp = new_frame_y
        
        if columns < PatchesShape[1]:
            if (columns%2)!=0:
                columns = columns-1
            
            new_frame_X = np.empty((rows,PatchesShape[1],slices))
            new_frame_y = np.empty((rows,PatchesShape[1],slices))
            
            for slc in range(slices):
                slicee_X = Xb_temp_X[:,:,slc]
                slicee_y = yb_temp[:,:,slc]
                
                for r in range(rows):
                    row_X = slicee_X[r,:]
                    row_y = slicee_y[r,:]
                    pad_arr_X = np.pad(row_X[:-1], (((PatchesShape[1]-columns)//2),), 'constant', constant_values=(0, 0))
                    pad_arr_y = np.pad(row_y[:-1], (((PatchesShape[1]-columns)//2),), 'constant', constant_values=(0, 0))
                    new_frame_X[r,:,slc] = pad_arr_X
                    new_frame_y[r,:,slc] = pad_arr_y
            
            Xb_temp = new_frame_X
            yb_temp = new_frame_y
        
        for p in range(Npatches):

            rows, columns, slices= Xb_temp.shape
            x = np.random.randint(rows-PatchesShape[0]+1) 
            y = np.random.randint(columns-PatchesShape[1]+1)
            z = np.random.randint(slices-PatchesShape[2]+1)  

            X_patches[i] = Xb_temp[x:x+PatchesShape[0], y:y+PatchesShape[1], z:z+PatchesShape[2]]
            y_patches[i] = yb_temp[x:x+PatchesShape[0], y:y+PatchesShape[1], z:z+PatchesShape[2]]
            i += 1
      
      del Xb_temp, yb_temp

    return X_patches, y_patches



def flip3D(X, y):
    """
    Flip the 3D image respect one of the 3 axis chosen randomly
    """
    choice = np.random.randint(3)
    if choice == 0: # flip on x
        X_flip, y_flip = X[::-1, :, :], y[::-1, :, :]
    if choice == 1: # flip on y
        X_flip, y_flip = X[:, ::-1, :], y[:, ::-1, :]
    if choice == 2: # flip on z
        X_flip, y_flip = X[:, :, ::-1], y[:, :, ::-1]
        
    return X_flip, y_flip


def rotation3D(X, y):
    """
    Rotate a 3D image with alfa, beta and gamma degree respect the axis x, y and z respectively.
    The three angles are chosen randomly between 0-30 degrees
    """
    alpha, beta, gamma = np.pi*np.random.random_sample(3,)/6
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(alpha), -np.sin(alpha)],
                   [0, np.sin(alpha), np.cos(alpha)]])
    
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    
    R = np.dot(np.dot(Rx, Ry), Rz)
    
    X_rot = np.empty_like(X)
    X_rot[:,:,:] = affine_transform(X[:,:,:], R, offset=0, order=3, mode='constant')
    y_rot = affine_transform(y, R, offset=0, order=0, mode='constant')
    
    return X_rot, y_rot

def brightness(X, y):
    """
    Changing the brighness of a image using power-law gamma transformation.
    Gain and gamma are chosen randomly for each image channel.
    
    Gain chosen between [0.9 - 1.1]
    Gamma chosen between [0.9 - 1.1]
    
    new_im = gain * im^gamma
    """
    
    X_new = np.zeros(X.shape)
    im = X[:,:,:]        
    gain, gamma = (1.2 - 0.8) * np.random.random_sample(2,) + 0.8
    im_new = np.sign(im)*gain*(np.abs(im)**gamma)
    X_new[:,:,:] = im_new 
    
    del im,im_new

    return X_new, y

def elastic(X, y):
    """
    Elastic deformation on a image and its target
    """  
    [Xel, yel] = elasticdeform.deform_random_grid([X, y], sigma=2, axis=[(0, 1, 2), (0, 1, 2)], order=[1, 0], mode='constant')
    
    return Xel, yel

def random_decisions(N):
    """
    Generate N random decisions for augmentation
    N should be equal to the batch size
    """
    
    decisions = np.zeros((N, 4)) # 4 is number of aug techniques to combine (patch extraction excluded)
    for n in range(N):
        decisions[n] = np.random.randint(2, size=4)
        
    return decisions

def combine_aug(X, y, do):
    """
    Combine randomly the different augmentation techniques written above
    """
    Xnew, ynew = X, y
    
    # make sure to use at least the 25% of original images
    if np.random.random_sample()>0.75:
        return Xnew, ynew
    
    else:   
        if do[0] == 1:
#             Xnew, ynew = flip3D(Xnew, ynew)
            Xnew, ynew = elastic(Xnew, ynew)

        if do[1] == 1:
#             Xnew, ynew = brightness(Xnew, ynew)
            Xnew, ynew = elastic(Xnew, ynew)

        if do[2] == 1:
#             Xnew, ynew = rotation3D(Xnew, ynew)
#             Xnew, ynew = flip3D(Xnew, ynew)
            Xnew, ynew = elastic(Xnew, ynew)

        if do[3] == 1:
            Xnew, ynew = elastic(Xnew, ynew)
        
        return Xnew, ynew

def aug_batch(Xb, Yb):
    """
    Generate a augmented image batch 
    """
    batch_size = len(Xb)
    newXb, newYb = np.empty_like(Xb), np.empty_like(Yb)
    
    decisions = random_decisions(batch_size)            
    inputs = [(X, y, do) for X, y, do in zip(Xb, Yb, decisions)]
    pool = mp.Pool(processes=8)
    multi_result = pool.starmap(combine_aug, inputs)
    pool.close()
    
    for i in range(len(Xb)):
        newXb[i], newYb[i] = multi_result[i][0], multi_result[i][1]

    del inputs,decisions   
    return newXb, newYb 

# LOAD DATA

In [ ]:
#load les imags et appliquer les augmentation

def load_img(img_files):
    ''' Load one image and its target form file
    '''
    N = len(img_files)
    # target
    y = nib.load(img_files[N-1]).get_fdata(dtype='float32', caching='unchanged')
    y[y==2]=1
    y[y==3]=1
    
    # y = y[0:630,130:590,0:35] #(630,460,35)
    y_les = nib.load(img_files[1]).get_fdata(dtype='float32', caching='unchanged')
    
    
    X_norm = np.empty(y.shape)

    X = nib.load(img_files[0]).get_fdata(dtype='float32', caching='unchanged')
    # X = X[0:630,130:590,0:35] 
    X = X*y
    lung = X[X!=0] 
    lung_norm = np.zeros_like(X) # background at -100

    
    xmax, xmin = np.max(lung), np.min(lung)
    lung_norm[X!=0]  = (lung - xmin)/(xmax - xmin)
    
    X_norm[:,:,:] = lung_norm        
            
    del(X, lung, lung_norm)
    
    X_norm = np.rot90(np.array(X_norm))
    y = np.rot90(np.array(y))
    y_les = np.rot90(np.array(y_les))
    
    return X_norm, y_les
    
    
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=4, n_channels=1, n_classes=1, shuffle=True, augmentation=False, patch_shape=(128,128,32), n_patches=1):
        'Initialization'
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.patch_shape = patch_shape
        self.n_patches = n_patches
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_IDs) // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data     
        X, y = self.__data_generation(list_IDs_temp)
        if self.augmentation == True:
            X, y = self.__data_augmentation(X, y)
        
        if index == self.__len__()-1:
            self.on_epoch_end()
        
        new_X = np.empty((self.batch_size*self.n_patches*self.patch_shape[2], self.patch_shape[0], self.patch_shape[1], 1))
        new_y = np.empty((self.batch_size*self.n_patches*self.patch_shape[2], self.patch_shape[0], self.patch_shape[1], 2))
        
        p = 0
        for n in range(len(X)):
            Xb = X[n,:,:,:,:]
            yb = y[n,:,:,:,:]
            
            for i in range(Xb.shape[2]):
                new_X[p,:,:,:]= Xb[:,:,i,:]
                new_y[p,:,:,:]= yb[:,:,i,:]
                p = p+1
        
        del X,y,Xb,yb
        return new_X, new_y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
  
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []
        for i, IDs in enumerate(list_IDs_temp):
          
          X_temp, y_temp = load_img(IDs)
          X.insert(i,X_temp)
          y.insert(i,y_temp)
        
        del X_temp, y_temp

        return X, y
        
    def __data_augmentation(self, X, y):
        'Apply augmentation'
        X_p, y_p = patch_extraction(X, y, PatchesShape=self.patch_shape, Npatches=self.n_patches)
        
        X_augg, y_augg = aug_batch(X_p, y_p)
        
        X_aug = np.empty(X_augg.shape)
        y_aug = np.empty(y_augg.shape)

        for b in range(len(y_aug)):
          y1 = y_augg[b,:]
          y2 = y_p[b,:]
          
          bgrd_ratio_after_aug = 1 - np.sum(y1[y1!=0])/(self.patch_shape[0] * self.patch_shape[1] * self.patch_shape[2])
          bgrd_ratio_before_aug = 1 - np.sum(y2[y2!=0])/(self.patch_shape[0] * self.patch_shape[1] * self.patch_shape[2])

          if (bgrd_ratio_after_aug > bgrd_ratio_before_aug) and (False):
            print(f'Index : {b} - Bg_ratio_BEFORE : {bgrd_ratio_before_aug} <<< Bg_ratio_AFTER : {bgrd_ratio_after_aug} ==> Drop augmentation.')
            X_aug[b,:] = X_p[b,:]
            y_aug[b,:] = y_p[b,:]
          else:
            X_aug[b,:] = X_augg[b,:]
            y_aug[b,:] = y_augg[b,:]


        X_new_shape = np.empty((self.batch_size*self.n_patches, *(self.patch_shape), 1))
        

        for n in range(len(X_aug)):
          image_data = X_aug[n,:,:,:]
          neww1 = np.zeros(image_data.shape)
#           neww2 = np.zeros(image_data.shape)
          for i in range(len(image_data[0,0,:])):
            slicee = image_data[:,:,i]
            slicee = np.uint8(slicee*255) 
            clahe1 = cv.createCLAHE(clipLimit=3.0)
            clahe_img1 = clahe1.apply(slicee)
            neww1[:,:,i]= clahe_img1

            
          X_new_shape[n,:,:,:,0] = neww1
        
        del X_aug,X_augg,y_augg,X_p,y_p,y1,y2,image_data,neww1,slicee,clahe_img1

        return X_new_shape.astype('float32'), to_categorical(y_aug).astype('float32')

# MODEL


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Conv3D, Conv2D, Conv3DTranspose, Dropout, ReLU, LeakyReLU, Concatenate, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import tensorflow_addons as tfa
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.layers import BatchNormalization

In [ ]:
#MODELS Unet3DGan
def double_conv(layer, Nf, ks, norm=True):
      x=layer
      del layer
      for ss in range(2):
          x = Conv2D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
          
          #a chaque etape effectuer la convolution afin d obtenir shape/2 avec nombre de filtre comme 4 eme D
          if (norm):
              x = BatchNormalization()(x) #Normaliser l image seulement 
          x = ReLU()(x) #Activer la sortie
      return x   

def Generator():
    '''
    Generator model
    '''
    
    def encoder_step(layer, Nf, ks, norm=True):
        x = double_conv(layer, Nf/2, ks, norm)
        del layer
        x = Conv2D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(x)  
        x = Dropout(0.1)(x)

        return x

    def bottlenek(layer, Nf, ks):
        x = Conv2D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        
        for i in range(4):
            y = Conv2D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
            x = BatchNormalization()(y)
            x = ReLU()(x)
            x = Concatenate()([x, y])
#             print(x.shape)
        return x

    def decoder_step(layer, layer_to_concatenate, Nf, ks):
        x = Dropout(0.1)(layer)
        del layer
        x = Conv2DTranspose(Nf, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal')(x)
        x = Concatenate()([x, layer_to_concatenate])
        x = double_conv(x, Nf, ks)
        return x

    layers_to_concatenate = []
    inputs = Input((512,512,1), name='input_image')
    Nfilter_start = 24
    depth = 5
    ks = 3
    x = inputs

    # encoder
    for d in range(depth):
        if d==0: 
            x = Conv2D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
            x = Conv2D(Nfilter_start, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
        else:
            x = encoder_step(x, Nfilter_start*np.power(2,d), ks)
        
        layers_to_concatenate.append(x)

    # bottlenek
    x = bottlenek(x, Nfilter_start*np.power(2,depth-1), ks)
    
    # decoder
    for d in range(depth-2, -1, -1): 
        x = decoder_step(x, layers_to_concatenate.pop(), Nfilter_start*np.power(2,d), ks)
    
    # classifier
    last = Conv2DTranspose(2, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal', activation='softmax', name='output_generator')(x)
    del layers_to_concatenate,x
    return Model(inputs=inputs, outputs=last, name='Generator')

def Discriminator():
    '''
    Discriminator model
    '''

    inputs = Input((512,512,1), name='input_image')
    targets = Input((512,512,2), name='target_image')
    Nfilter_start = 24
    depth = 3
    ks = 3

    def encoder_step(layer, Nf, norm=True):
        x = Conv2D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        if norm:
            x = InstanceNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.1)(x)
        
        return x

    x = Concatenate()([inputs, targets])

    for d in range(depth+1):
        if d==0:
            x = encoder_step(x, Nfilter_start*np.power(2,d), False)
        else:
            x = encoder_step(x, Nfilter_start*np.power(2,d))
            

    last = Conv2D(1, ks, strides=1, padding='valid', kernel_initializer='he_normal', name='output_discriminator')(x) 
    
    del x
    return Model(inputs=[inputs,targets], outputs=last, name='Discriminator')

# LOSSES

In [ ]:
#METRICS EVALUATING
import tensorflow.compat.v1 as tff
import numpy as np
from scipy import ndimage

def lossa(y_true,y_pred):
    smooth = 1e-5  
    y_true = tf.convert_to_tensor(y_true)
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(y_true * y_pred, axis=[0,1,2])
    union = K.sum(y_true, axis=[0,1,2]) + K.sum(y_pred, axis=[0,1,2])
    dice =1- K.mean((2. * intersection + smooth)/(union + smooth))
    return dice

def dice_coef_1(y_true, y_pred):
   
    y_true = tf.convert_to_tensor(y_true)
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred)
    y_pred = tf.cast(y_pred, tf.float32)
    class_weights = [1.0,1.0]
    num = tf.math.reduce_sum(tf.math.multiply(class_weights, tf.math.reduce_sum(tf.math.multiply(y_true, y_pred), axis=[0,1,2])))
    den = tf.math.reduce_sum(tf.math.multiply(class_weights, tf.math.reduce_sum(tf.math.add(y_true, y_pred), axis=[0,1,2])))+1e-5

    return 1-2*num/den


def dice_coef(y_true, y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    smooth = 1e-5

    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def binary_dice3d(s,g):
    #dice score of two 3D volumes
    smooth = 1e-5
    num=np.sum(np.multiply(s, g))
    denom=s.sum() + g.sum() + smooth
    
    return  2.0*num/denom

def sensitivity (seg,ground): 
    #computs false negative rate
    smooth = 1e-5
    num=np.sum(np.multiply(ground, seg))
    denom=np.sum(ground)+smooth
    
    return  num/denom

def specificity (seg,ground): 
    #computes false positive rate
    smooth = 1e-5
    num=np.sum(np.multiply(ground==0, seg ==0))
    denom=np.sum(ground==0)+smooth
    
    return  num/denom

def border_map(binary_img,neigh):
    """
    Creates the border for a 3D image
    """
    binary_map = np.asarray(binary_img, dtype=np.uint8)
    neigh = neigh
    west = ndimage.shift(binary_map, [-1, 0,0], order=0)
    east = ndimage.shift(binary_map, [1, 0,0], order=0)
    north = ndimage.shift(binary_map, [0, 1,0], order=0)
    south = ndimage.shift(binary_map, [0, -1,0], order=0)
    top = ndimage.shift(binary_map, [0, 0, 1], order=0)
    bottom = ndimage.shift(binary_map, [0, 0, -1], order=0)
    cumulative = west + east + north + south + top + bottom
    border = ((cumulative < 6) * binary_map) == 1
    return border

def border_distance(ref,seg):
    """
    This functions determines the map of distance from the borders of the
    segmentation and the reference and the border maps themselves
    """
    neigh=8
    border_ref = border_map(ref,neigh)
    border_seg = border_map(seg,neigh)
    oppose_ref = 1 - ref
    oppose_seg = 1 - seg
    # euclidean distance transform
    distance_ref = ndimage.distance_transform_edt(oppose_ref)
    distance_seg = ndimage.distance_transform_edt(oppose_seg)
    distance_border_seg = border_ref * distance_seg
    distance_border_ref = border_seg * distance_ref
    return distance_border_ref, distance_border_seg#, border_ref, border_seg

def Hausdorff_distance(ref,seg):
    """
    This functions calculates the average symmetric distance and the
    hausdorff distance between a segmentation and a reference image
    :return: hausdorff distance and average symmetric distance
    """
    ref_border_dist, seg_border_dist = border_distance(ref,seg)
    hausdorff_distance = np.max([np.max(ref_border_dist), np.max(seg_border_dist)])
    return hausdorff_distance

In [ ]:
#calcul losses

def discriminator_loss(disc_real_output, disc_fake_output):
    real_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_real_output) - disc_real_output, 2))
    fake_loss = tf.math.reduce_mean(tf.math.pow(tf.zeros_like(disc_fake_output) - disc_fake_output, 2))

    disc_loss = 0.5*(real_loss + fake_loss)

    return disc_loss


def generator_loss(target, gen_output, disc_fake_output, alpha):
    
    # generalized dice loss
    dice_loss = lossa(target, gen_output)

    # disc loss
    disc_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_fake_output) - disc_fake_output, 2))
       
    # total loss
    gen_loss = alpha*dice_loss + disc_loss

    return gen_loss, dice_loss, disc_loss

# TRAIN


In [ ]:
from sklearn.metrics import precision_score
#Importing tqdm function of tqdm module 
from tqdm import tqdm  
from time import sleep 
import math
from keras import backend as K


# Models
G = Generator()
D = Discriminator()

if os.path.exists(PATH + f'/{VERSION}_Generator.h5')==True and os.path.exists(PATH + f'/{VERSION}_Discriminator.h5')==True: 
    G.load_weights(PATH + f'/{VERSION}_Generator.h5')
    D.load_weights(PATH + f'/{VERSION}_Discriminator.h5')

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-3, beta_1=0.5)

@tf.function
def train_step(image, target, alpha):
  
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      gen_output = G(image, training=True)
      disc_real_output = D([image, target], training=True)
      disc_fake_output = D([image, gen_output], training=True)
      disc_loss = discriminator_loss(disc_real_output, disc_fake_output)

    
      gen_loss, dice_loss, disc_loss_gen = generator_loss(target, gen_output, disc_fake_output, alpha)
      
    
      del gen_output, disc_real_output, disc_fake_output,image
  generator_gradients = gen_tape.gradient(gen_loss, G.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss, D.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, G.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, D.trainable_variables))
      
  return gen_loss, dice_loss, disc_loss_gen
        
@tf.function
def test_step(image, target, alpha):
    gen_output = G(image, training=False)

    disc_real_output = D([image, target], training=False)
    disc_fake_output = D([image, gen_output], training=False)
    disc_loss = discriminator_loss(disc_real_output, disc_fake_output)
    
    gen_loss, dice_loss, disc_loss_gen = generator_loss(target, gen_output, disc_fake_output, alpha)
    del gen_output, disc_real_output, disc_fake_output,image    
    return gen_loss, dice_loss, disc_loss_gen

def fit(train_gen, valid_gen, alpha, epochs, jump):
    
    if os.path.exists(PATH)==False:
        os.mkdir(PATH)
        
    Nt = len(train_gen)
    history = {'train': [], 'valid': []}
    
    #stats={'dice':[],'spec':[],'sen':[],'hau95':[]}
    prev_loss = np.inf
    
    epoch_ugan_loss = tf.keras.metrics.Mean()
    epoch_dice_loss = tf.keras.metrics.Mean()
    epoch_disc_loss = tf.keras.metrics.Mean()
    epoch_ugan_loss_val = tf.keras.metrics.Mean()
    epoch_dice_loss_val = tf.keras.metrics.Mean()
    epoch_disc_loss_val = tf.keras.metrics.Mean()
    
    vv_min = 2
    
    for e in range(epochs):
        print('Epoch {}/{}'.format(e+1,epochs))
        b = 0
        
#         x=(EXEC -1)*EPOCHS + e
#         lrg=(math.exp(-x))*0.001 + 0.00003
#         lrd=(math.exp(-x))*0.001 + 0.00003
#         K.set_value(generator_optimizer.learning_rate, lrg)
#         K.set_value(discriminator_optimizer.learning_rate, lrd)
        
        for i in tqdm(range(len(train_gen))):
            _batch = train_gen[i]
            Xb = _batch[0]
            yb = _batch[1]
            b += 1 
            losses = train_step(Xb, yb, alpha)
            epoch_ugan_loss.update_state(losses[0])
            epoch_dice_loss.update_state(losses[1])
            epoch_disc_loss.update_state(losses[2])

            print('\rBatch: {}/{} - loss: {:.4f} - dice_loss: {:.4f} - disc_loss: {:.4f}'
                        .format(b, Nt, epoch_ugan_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()))

          
        del Xb,yb,_batch   
        history['train'].append([epoch_ugan_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()])

        
        
        for i in tqdm(range(len(valid_gen))):
            _batch = valid_gen[i]
            Xb = _batch[0]
            yb = _batch[1]
            losses_val = test_step(Xb, yb, alpha)
            epoch_ugan_loss_val.update_state(losses_val[0])
            epoch_dice_loss_val.update_state(losses_val[1])
            epoch_disc_loss_val.update_state(losses_val[2])

             
        print('\n               loss_val: {:.4f} - dice_loss_val: {:.4f} - disc_loss_val: {:.4f}'
                     .format(epoch_ugan_loss_val.result(), epoch_dice_loss_val.result(), epoch_disc_loss_val.result()))
        
        history['valid'].append([epoch_ugan_loss_val.result(), epoch_dice_loss_val.result(), epoch_disc_loss_val.result()])
#         print(history)
        # save pred image at epoch e 
     
        y_pred = G.predict(Xb)
        y_true = np.argmax(yb, axis=-1)
        y_pred = np.argmax(y_pred, axis=-1)
        
        canvas = np.zeros((512,512*3))
        idx = np.random.randint(len(Xb))
        
        x = Xb[idx,:,:,0] 
        # print('image shape: ',x.shape)
        canvas[0:512, 0:512] = (x - np.min(x))/(np.max(x)-np.min(x)+1e-6)
        canvas[0:512, 512:2*512] = y_true[idx,:,:]/3
        canvas[0:512, 2*512:3*512] = y_pred[idx,:,:]/3
        
        if jump:
            n_e = e + (EXEC-1)*epochs
        else:
            n_e = e
            
        fname = (PATH + f'/{VERSION}'+'_pred@epoch_{:03d}.png').format(n_e+1)
        mpim.imsave(fname, canvas, cmap='gray')
        # save models
        
        try :
            if epoch_dice_loss_val.result() <= vv_min:
                G.save_weights(PATH + f'/{VERSION}_Generator.h5') 
                D.save_weights(PATH + f'/{VERSION}_Discriminator.h5')
                vv_min = epoch_dice_loss_val.result()
        except:
            print('Models not saved')
        
        # resets losses states
        epoch_v2v_loss.reset_states()
        epoch_dice_loss.reset_states()
        epoch_disc_loss.reset_states()
        epoch_v2v_loss_val.reset_states()
        epoch_dice_loss_val.reset_states()
        epoch_disc_loss_val.reset_states()
        
        del(Xb, yb, _batch, canvas, y_pred, y_true, idx)
       
    return history

# MAIN 

In [ ]:
# create the training and validation sets


BATCH_SIZE = 1
ALPHA = 5
EPOCHS = 8
NB_CLASSES = 2
N_PATCHES = 1
PATCH_SHAPE =(512,512,32)


JUMP = False
pred_img = (PATH + f'/{VERSION}'+'_pred@epoch_{:03d}.png').format((EXEC-1)*EPOCHS) #0,8,16
if os.path.exists(pred_img)==True:
    JUMP = True



ct_list = sorted(glob.glob('../input/covid19-ct-scans/ct_scans/*.nii'))
seg_list = sorted(glob.glob('../input/covid19-ct-scans/lung_and_infection_mask/*.nii'))
seg_les_list = sorted(glob.glob('../input/covid19-ct-scans/infection_mask/*.nii'))




Nim = len(ct_list)
print(f'len dataset : {Nim}')
idx = np.arange(Nim)

# idxTrain, idxValid = train_test_split(idx, test_size=0.20) #0.20 a la place de 0.25
idxTrain = [15, 3, 14, 16, 4, 2, 17, 6, 0, 1, 9, 8, 10, 11, 12]
idxValid = [7, 18, 13, 19]




print(f'len train : {len(idxTrain)}, len test : {len(idxValid)}')

sets = {'train': [], 'valid': []}

for i in idxTrain:
    sets['train'].append([ct_list[i], seg_les_list[i], seg_list[i]])
for i in idxValid:
    sets['valid'].append([ct_list[i], seg_les_list[i], seg_list[i]])
      
del ct_list,seg_list,idx,Nim,idxTrain

train_gen = DataGenerator(sets['train'], batch_size=BATCH_SIZE, patch_shape=PATCH_SHAPE, n_patches=N_PATCHES, n_classes=NB_CLASSES, augmentation=True)
valid_gen = DataGenerator(sets['valid'], batch_size=BATCH_SIZE, patch_shape=PATCH_SHAPE, n_patches=N_PATCHES, n_classes=NB_CLASSES, augmentation=True)

del sets

In [ ]:
%%capture cap --no-stderr
h = fit(train_gen, valid_gen, ALPHA, EPOCHS, JUMP) 

In [ ]:
with open(PATH + f'/{VERSION}_fit_output_{EXEC}.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
%%capture cap_h --no-stderr
print(h)

In [ ]:
with open(PATH + f'/{VERSION}_h_{EXEC}.txt', 'w') as f:
    f.write(cap_h.stdout)

In [ ]:
#last loop for metrics
from sklearn.metrics import precision_score
#Importing tqdm function of tqdm module 
from tqdm import tqdm  
from time import sleep 


G = Generator()

G.load_weights(PATH + f'/{VERSION}_Generator.h5')

stats={'dice':[],'spec':[],'sen':[]}
i=0
list_id = []

for bb in tqdm(range(len(valid_gen))):
    for k in range(BATCH_SIZE):
        for n in range(N_PATCHES*32):
            list_id.append(idxValid[i])
        i=i+1

i=0
for Xb,yb in tqdm(valid_gen):
    for k in range(len(Xb)):
#         print(len(Xb))
#         print(i)
        segG = G.predict(Xb)[k,:]
        segTr = yb[k,:]
        case = list_id[i]
#         case = 0
        i=i+1
        segG_all = np.argmax(segG, axis=-1)
        segTr_all = np.argmax(segTr, axis=-1)
#         print(np.min(segG_all),np.max(segG_all))
        dsc = dice_coef(segG_all>0,segTr_all>0)

        spec = specificity(segG_all>0,segTr_all>0)

        sen = sensitivity(segG_all>0,segTr_all>0)

#         hau= Hausdorff_distance(segG_all>0,segTr_all>0)

#         stats['dice'].append({'caseID':int(case),'value':dsc})
        stats['dice'].append({'caseID':int(case),'value':dsc})
        stats['spec'].append({'caseID':int(case),'value':spec})
        stats['sen'].append({'caseID':int(case),'value':sen})
#         stats['hau95'].append({'caseID':int(case),'value':hau})

        del segG,segG_all,segTr,segTr_all

with open(PATH + f'/{VERSION}_stats_{EXEC}.json', 'w') as f:
          json.dump(stats, f, indent=2)
          print("stats saved.")

In [ ]:
%%capture cap_metrics --no-stderr
dice = []
sen = []
spec = []
hau = []

import numpy as np

# print (stats['dice'][0]['WT'])
for k in range(len(valid_gen)*BATCH_SIZE*N_PATCHES*32):
    dice.append(stats['dice'][k]['value'])
    sen.append(stats['sen'][k]['value'])
    spec.append(stats['spec'][k]['value'])
#     hau.append(stats['hau95'][k]['value'])

print(f'DSC: {np.mean(dice)}')
print(f'SEN: {np.mean(sen)}')
print(f'SPE: {np.mean(spec)}')
# print(f'H95: {np.mean(hau)}')


In [ ]:
with open(PATH + f'/{VERSION}_metrics_{EXEC}.txt', 'w') as f:
    f.write(cap_metrics.stdout)

In [ ]:
! zip -r './results.zip' {PATH+'/*'}

In [ ]:
from IPython.display import FileLink
FileLink(r'./results.zip')
# then in colab : !wget "https://....kaggle.net/...../results.zip"